<a href="https://colab.research.google.com/github/thisissamuca/GOES_16/blob/main/APPLIED_MASK_CLOUD.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install rasterio netCDF4 satpy xarray opencv-python

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.4/42.4 kB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.6/199.6 kB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 22.2/22.2 MB 43.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.3/9.3 MB 58.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 45.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.7/11.7 MB 55.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.3/84.3 kB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.7/4.7 MB 58.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 455.9/455.9 kB 24.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 658.4/658.4 kB 30.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 47.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 118.6/118.6 kB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 255.4/255.

In [ ]:
import os
import numpy as np
import rasterio
import xarray as xr
import cv2
from typing import List, Tuple, Optional, Dict
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
def extract_key(filename: str) -> Optional[str]:
    """
    Extrai o valor entre 'G16_s' e '_e' do nome do arquivo.
    """
    try:
        start = filename.index('G16_s') + len('G16_s')
        end = filename.index('_e', start)
        return filename[start:end]
    except ValueError:
        return None


def match_files(mask_files: List[str], scene_files: List[str]) -> List[Tuple[str, str]]:
    """
    Encontra pares de arquivos correspondentes baseados em suas chaves.
    """
    mask_dict: Dict[str, str] = {}
    for mask_file in mask_files:
        if key := extract_key(mask_file):
            mask_dict[key] = mask_file

    matches = []
    for scene_file in scene_files:
        if (key := extract_key(scene_file)) and (key in mask_dict):
            matches.append((mask_dict[key], scene_file))

    return matches


def build_output_filename(scene_path: str) -> str:
    """
    Constrói o nome do arquivo de saída baseado no padrão do arquivo de cena.

    Formato: parte1_parte2_parte3_parte4_parte5.tif
    Remove a extensão .nc se existir e substitui por .tif
    """
    filename = os.path.basename(scene_path)

    # Remove a extensão .nc se existir
    filename = os.path.splitext(filename)[0]

    parts = filename.split('_')

    if len(parts) >= 6:
        # Pega as primeiras 5 partes após o prefixo
        base_name = '_'.join(parts[1:6])
    else:
        # Fallback: usa o nome do arquivo sem extensão
        base_name = filename

    return f"{base_name}.tif"


def process_and_export(mask_path: str, scene_path: str, output_dir: str) -> None:
    """
    Processa arquivos de máscara e cena e exporta com nome padronizado.
    """
    # Verifica existência dos arquivos
    if not all(os.path.exists(p) for p in (mask_path, scene_path)):
        raise FileNotFoundError("Um ou mais arquivos de entrada não existem")

    # Gera nome do arquivo de saída
    output_filename = build_output_filename(scene_path)
    output_path = os.path.join(output_dir, output_filename)

    # Processa a máscara
    with rasterio.open(mask_path) as src:
        mask_data = src.read(1)
        profile = src.profile.copy()
        mask_data = np.where(mask_data == 0, np.nan, mask_data)

    # Processa a cena
    with xr.open_dataset(scene_path) as ds:
        first_var = next(iter(ds.variables))
        scene_data = ds[first_var].data

    # Redimensiona a cena
    target_height, target_width = mask_data.shape
    scene_data = cv2.resize(
        scene_data,
        (target_width, target_height),
        interpolation=cv2.INTER_LINEAR
    )

    # Aplica operação
    result = scene_data * mask_data

    # Configurações de saída
    profile.update({
        'driver': 'GTiff',
        'count': 1,
        'dtype': result.dtype,
        'compress': 'lzw',
        'tiled': True
    })

    # Cria diretório de saída se necessário
    os.makedirs(os.path.dirname(output_path), exist_ok=True)

    # Exporta resultado
    with rasterio.open(output_path, 'w', **profile) as dst:
        dst.write(result, 1)


def batch_process(
    mask_files: List[str],
    scene_files: List[str],
    output_dir: str,
    start_index: int = 0,
    verbose: bool = True
) -> None:
    """
    Processa em lote os pares de arquivos correspondentes a partir de um índice específico.

    Args:
        mask_files: Lista de caminhos para arquivos .tif (máscaras)
        scene_files: Lista de caminhos para arquivos .nc (cenas)
        output_dir: Diretório de saída para os resultados
        start_index: Índice do par a partir do qual iniciar o processamento
        verbose: Se True, imprime progresso
    """
    matches = match_files(mask_files, scene_files)

    if not matches:
        print("Nenhum par correspondente encontrado para processamento.")
        return

    if verbose:
        print(f"Encontrados {len(matches)} pares correspondentes para processamento.")

    # Processa apenas a partir do start_index
    for i, (mask_file, scene_file) in enumerate(matches[start_index:], start_index + 1):
        try:
            process_and_export(mask_file, scene_file, output_dir)
            if verbose:
                print(f'Processado {i}/{len(matches)}: {os.path.basename(scene_file)} → {build_output_filename(scene_file)}')
        except Exception as e:
            print(f"Erro processando {mask_file} e {scene_file}: {str(e)}")


# Exemplo de uso para processar a partir do par 0
if __name__ == "__main__":
    mask_dir = '/content/drive/MyDrive/NBR/ARQUIVOS_PROCESSADOS/MASCARA_RECLASSIFICADA'
    scene_dir = '/content/drive/MyDrive/NBR/ARQUIVOS_BRUTOS/ABI-L2-CMIPF/netCDF'
    output_dir = '/content'

    # Lista arquivos garantindo que sejam arquivos (não diretórios)
    mask_files = [
        os.path.join(mask_dir, f)
        for f in os.listdir(mask_dir)
        if os.path.isfile(os.path.join(mask_dir, f)) and f.endswith('.tif')
    ]

    scene_files = [
        os.path.join(scene_dir, f)
        for f in os.listdir(scene_dir)
        if os.path.isfile(os.path.join(scene_dir, f)) and f.endswith('.nc')
    ]

    # Verifica se há arquivos para processar
    if not mask_files or not scene_files:
        print("Nenhum arquivo encontrado em um ou ambos os diretórios.")
    else:
        batch_process(mask_files, scene_files, output_dir, start_index=0, verbose=True)